In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab, sparse_to_doc, connections_pixel_list_to_tab
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
from datetime import datetime,timedelta

In [2]:
plate = 13
date_init = datetime(2020,7,1,19,57)

In [3]:
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(3)]

In [4]:
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [5]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [6]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [7]:
skeleton_docs=[sparse_to_doc(sparse_skel) for sparse_skel in dic_skels]

In [8]:
nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(realign(skeleton_doc,skeleton_docs[0],2))) for skeleton_doc in skeleton_docs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

number_common_nodes_found : 2143 [[ 1.00000000e+00 -1.38777878e-17]
 [ 1.38777878e-17  1.00000000e+00]] [0.0000000e+00 1.8189894e-12]
100000
200000
300000
400000
500000
600000
866490 867437
105736 740
379 109
86 70
54 39
30 0
0 0
number_common_nodes_found : 477 [[ 9.99999998e-01  5.78173765e-05]
 [-5.78173765e-05  9.99999998e-01]] [14.83091045 -2.20804855]
100000
200000
300000
400000
500000
600000
number_common_nodes_found : 562 [[ 9.99999999e-01 -4.53206814e-05]
 [ 4.53206814e-05  9.99999999e-01]] [ 1.22085997 -1.27077265]
100000
200000
300000
400000
500000
600000
899840 901142
110361 838
515 225
198 169
43 40
32 0
0 0
number_common_nodes_found : 330 [[ 9.99999752e-01 -7.04457346e-04]
 [ 7.04457346e-04  9.99999752e-01]] [ 27.99958833 -11.29952894]
100000
200000
300000
400000
500000
600000
number_common_nodes_found : 456 [[ 9.99999919e-01 -4.02646068e-04]
 [ 4.02646068e-04  9.99999919e-01]] [10.82977501 -7.95353488]
100000
200000
300000
400000
500000
600000
number_common_nodes_found : 

In [9]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-1
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,-1,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 1
number removed first phase 99
number removed second phase 470
3 0 [9658 6714]
3 1 [9662 6724]
i= 0
number removed first phase 130
number removed second phase 498
2 0 [9661 6729]
2 1 [9658 6714]
2 2 [9662 6724]


In [10]:
nx_graph_pruned=downstream_graphs
poss_aligned=downstream_poss

In [11]:
# for i, date in enumerate(dates):
#     sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [14]:
for i, date in enumerate(dates):
    from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled_matlab.csv'))
    from_nx_to_tab_matlab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
#     tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

In [23]:
# for i, date in enumerate(dates):
#     tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
#     tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')

In [ ]:
connections_growth_pattern = [whole_movement_identification(nx_graph_pruned[i],nx_graph_pruned[i+1],poss_aligned[i],poss_aligned[i+1]) for i in range(len(dates)-1)]
tip_origins = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]
for i in range(len(tabs_connection)):
    connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(get_path(dates[i],plate,True,extension='_connection.csv'))

0.0
0.125


In [ ]:
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [ ]:
xbegin=14000
xend = 18000
ybegin = 8000
yend=16000
sub_nx_graphs = [sub_graph(nx_graph,poss_aligned[i],xbegin,xend,ybegin,yend) for i, nx_graph in enumerate(nx_graph_pruned)]
# sub_skeleton= [generate_skeleton(nx_graph,(xend-xbegin,yend-ybegin),(xbegin,ybegin)) for nx_graph in sub_nx_graphs]

In [13]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[sub_nx_graphs[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(sub_nx_graphs[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 19
number removed second phase 36
116 0 [16887  8355]
116 1 [16893  8366]
i= 2
number removed first phase 13
number removed second phase 27
121 0 [17317  9088]
121 1 [17312  9091]
121 2 [17311  9107]
i= 1
number removed first phase 12
number removed second phase 32
119 0 [17318  9090]
119 1 [17317  9088]
119 2 [17312  9091]
119 3 [17311  9107]


In [65]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in downstream_graphs]

In [69]:
t=0
tp1=1
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],downstream_poss[t],downstream_poss[tp1],sub_skeleton[t+1].todense(),sub_skeleton[tp1+1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
skeleton_docs_cleaned= [generate_skeleton(nx_graph,skeleton_docs_aligned[i].shape) for i,nx_graph in enumerate(nx_graph_pruned)]